In [15]:
import pandas as pd

In [16]:
pd.set_option("max_columns",100)

In [17]:
tweets = pd.read_pickle("../../data/cp_tweets.pkl")

## have a lok at the data and apply preproccessing to it

In [18]:
import sys
lib_dir = "../../libs/"
if lib_dir not in sys.path:
    sys.path.append(lib_dir)

In [19]:
import preprocessor

In [20]:
tweets["clean_text"] = preprocessor.preprocess_tweets(tweets['text'],False,False)

In [21]:
tweets.head(10).text 

0         @theblaze @realDonaldTrump https://t.co/TY9DlZ...
1         @BarackObama  @FBI @LORETTALYNCH  ALL IN COLLU...
10        @mike4193496 @realDonaldTrump I TOTALLY CONCUR...
100       #morningjoe Alleged child rapist @realDonaldTr...
1000      @realDonaldTrump "Obama Founder of Isis Claim"...
10000     @grammybijou3 even after @realDonaldTrump said...
100000    @RadicalRW @mikemckenna19 @MMQC15 @WalshFreedo...
100001    West Bend.... @FoxNews | @realDonaldTrump 3:) ...
100002    @HillaryClinton @realDonaldTrump @MELANIATRUMP...
100003    @BigDuhie1955 Yep Scary times like you said Gl...
Name: text, dtype: object

In [22]:
# lower case text
tweets.head(10).clean_text 

0                                @theblaze @realdonaldtrump
1         @barackobama @fbi @lorettalynch collusion toge...
10        @realdonaldtrump totally concur election cra c...
100       #morningjoe alleged child rapist @realdonaldtr...
1000      @realdonaldtrump obama founder isis claim also...
10000     even @realdonaldtrump said sarcasm @barackobam...
100000    @radicalrw@walshfreedom @barackobama @hillaryc...
100001    west bend @foxnews@realdonaldtrump#trumpers bu...
100002    @hillaryclinton @realdonaldtrump @melaniatrump...
100003    yep scary times like said glad brothers made b...
Name: clean_text, dtype: object

## Stemming & Lammatization

In [ ]:
x.apply()

In [1]:
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import * 

In [2]:
stemmer = SnowballStemmer("english")

In [62]:
text_stem_lam = tweets.sample(100000).clean_text.apply(lambda x: x.split(' '))\
                                        .apply(lambda l:
list(map(
    lambda x: stemmer.stem(WordNetLemmatizer().lemmatize(x, pos='v')),
    l))
)


In [63]:
text_stem_lam[1:10]

23080     [man, control, kid, get, sit, want, man, ur, p...
177669    [@realdonaldtrump, @jerryjrfalwel, @washington...
596638    [@craigipedia, 👏🏼👏🏼👏🏼👏🏼@realdonaldtrump, frien...
339321    [@realdonaldtrump, justic, equalityopportunity...
419631    [#nacion, insist, #donaldtrump, en, la, constr...
445341    [love, busi, close, sunday, make, want, give, ...
498659    [want, vote, @realdonaldtrump, @hillaryclinton...
247817    [must, never, forgetit, hollywood, person, med...
20038     [@hillaryclinton, win, elect, will, without, d...
Name: clean_text, dtype: object

## Word Dictionary

In [67]:
#x = tweets.sample(100000).clean_text.apply(lambda x: x.split(' '))
#doc_clean = list(x)
doc_clean = text_stem_lam

In [68]:
# Importing Gensim
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index
dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

2018-10-03 00:57:01,701 : INFO : 'pattern' package not found; tag filters are not available for English
2018-10-03 00:57:01,721 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-10-03 00:57:02,007 : INFO : adding document #10000 to Dictionary(17667 unique tokens: ['#liar', '#taxfraud', '@nytim', '@realdonaldtrump', 'credibl']...)
2018-10-03 00:57:02,280 : INFO : adding document #20000 to Dictionary(29260 unique tokens: ['#liar', '#taxfraud', '@nytim', '@realdonaldtrump', 'credibl']...)
2018-10-03 00:57:02,549 : INFO : adding document #30000 to Dictionary(39009 unique tokens: ['#liar', '#taxfraud', '@nytim', '@realdonaldtrump', 'credibl']...)
2018-10-03 00:57:02,806 : INFO : adding document #40000 to Dictionary(48100 unique tokens: ['#liar', '#taxfraud', '@nytim', '@realdonaldtrump', 'credibl']...)
2018-10-03 00:57:03,087 : INFO : adding document #50000 to Dictionary(56575 unique tokens: ['#liar', '#taxfraud', '@nytim', '@realdonaldtrump', 'credibl']...)
2018-10-03 00:

## LDA Model

In [69]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

**NOTE** the higher the `passes` arg is the more time it will take to train. Im not sure if it guarentees better topics.

In [70]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=2, id2word = dictionary, passes=5)

2018-10-03 00:58:00,406 : INFO : using symmetric alpha at 0.5
2018-10-03 00:58:00,408 : INFO : using symmetric eta at 0.5
2018-10-03 00:58:00,434 : INFO : using serial LDA version on this node
2018-10-03 00:58:00,498 : INFO : running online (multi-pass) LDA training, 2 topics, 5 passes over the supplied corpus of 100000 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
2018-10-03 00:58:00,499 : INFO : PROGRESS: pass 0, at document #2000/100000
2018-10-03 00:58:04,205 : INFO : merging changes from 2000 documents into a model of 100000 documents
2018-10-03 00:58:04,280 : INFO : topic #0 (0.500): 0.056*"@realdonaldtrump" + 0.016*"@hillaryclinton" + 0.011*"#trump" + 0.007*"like" + 0.006*"say" + 0.005*"trump" + 0.005*"@foxnew" + 0.005*"go" + 0.005*"hillari" + 0.004*"#nevertrump"
2018-10-03 00:58:04,284 : INFO : topic #1 (0.500): 0.048*"@realdonaldtrump" + 0.024*"@hillaryclinton" + 0.014*"t

2018-10-03 00:58:26,266 : INFO : topic diff=0.180953, rho=0.301511
2018-10-03 00:58:26,269 : INFO : PROGRESS: pass 0, at document #24000/100000
2018-10-03 00:58:27,881 : INFO : merging changes from 2000 documents into a model of 100000 documents
2018-10-03 00:58:27,916 : INFO : topic #0 (0.500): 0.055*"@realdonaldtrump" + 0.014*"#trump" + 0.011*"@hillaryclinton" + 0.007*"like" + 0.006*"#nevertrump" + 0.006*"@cnn" + 0.005*"@foxnew" + 0.005*"#imwithh" + 0.004*"tax" + 0.004*"think"
2018-10-03 00:58:27,921 : INFO : topic #1 (0.500): 0.045*"@realdonaldtrump" + 0.028*"@hillaryclinton" + 0.015*"trump" + 0.008*"get" + 0.007*"say" + 0.006*"u" + 0.006*"hillari" + 0.006*"vote" + 0.006*"go" + 0.006*"peopl"
2018-10-03 00:58:27,923 : INFO : topic diff=0.178864, rho=0.288675
2018-10-03 00:58:27,926 : INFO : PROGRESS: pass 0, at document #26000/100000
2018-10-03 00:58:29,357 : INFO : merging changes from 2000 documents into a model of 100000 documents
2018-10-03 00:58:29,394 : INFO : topic #0 (0.500):

2018-10-03 00:58:45,182 : INFO : topic #1 (0.500): 0.047*"@realdonaldtrump" + 0.027*"@hillaryclinton" + 0.014*"trump" + 0.008*"get" + 0.007*"say" + 0.007*"hillari" + 0.006*"go" + 0.006*"vote" + 0.006*"know" + 0.005*"u"
2018-10-03 00:58:45,184 : INFO : topic diff=0.154134, rho=0.208514
2018-10-03 00:58:45,187 : INFO : PROGRESS: pass 0, at document #48000/100000
2018-10-03 00:58:46,358 : INFO : merging changes from 2000 documents into a model of 100000 documents
2018-10-03 00:58:46,394 : INFO : topic #0 (0.500): 0.053*"@realdonaldtrump" + 0.014*"#trump" + 0.011*"@hillaryclinton" + 0.007*"@cnn" + 0.007*"#imwithh" + 0.007*"#nevertrump" + 0.006*"@foxnew" + 0.005*"tax" + 0.005*"like" + 0.005*""
2018-10-03 00:58:46,396 : INFO : topic #1 (0.500): 0.047*"@realdonaldtrump" + 0.027*"@hillaryclinton" + 0.014*"trump" + 0.008*"get" + 0.007*"say" + 0.006*"hillari" + 0.006*"go" + 0.006*"know" + 0.006*"vote" + 0.005*"u"
2018-10-03 00:58:46,398 : INFO : topic diff=0.152083, rho=0.204124
2018-10-03 00:58

2018-10-03 00:59:00,633 : INFO : PROGRESS: pass 0, at document #70000/100000
2018-10-03 00:59:01,757 : INFO : merging changes from 2000 documents into a model of 100000 documents
2018-10-03 00:59:01,791 : INFO : topic #0 (0.500): 0.051*"@realdonaldtrump" + 0.014*"#trump" + 0.010*"@hillaryclinton" + 0.008*"@cnn" + 0.007*"#imwithh" + 0.006*"#nevertrump" + 0.005*"tax" + 0.005*"@foxnew" + 0.004*"#hillaryclinton" + 0.004*""
2018-10-03 00:59:01,793 : INFO : topic #1 (0.500): 0.048*"@realdonaldtrump" + 0.026*"@hillaryclinton" + 0.013*"trump" + 0.008*"get" + 0.007*"say" + 0.007*"hillari" + 0.006*"go" + 0.006*"u" + 0.005*"vote" + 0.005*"like"
2018-10-03 00:59:01,797 : INFO : topic diff=0.137187, rho=0.169031
2018-10-03 00:59:01,799 : INFO : PROGRESS: pass 0, at document #72000/100000
2018-10-03 00:59:02,880 : INFO : merging changes from 2000 documents into a model of 100000 documents
2018-10-03 00:59:02,922 : INFO : topic #0 (0.500): 0.051*"@realdonaldtrump" + 0.014*"#trump" + 0.010*"@hillarycl

2018-10-03 00:59:15,397 : INFO : topic #1 (0.500): 0.050*"@realdonaldtrump" + 0.026*"@hillaryclinton" + 0.014*"trump" + 0.007*"say" + 0.007*"get" + 0.006*"go" + 0.006*"hillari" + 0.006*"like" + 0.005*"vote" + 0.005*"u"
2018-10-03 00:59:15,399 : INFO : topic diff=0.127971, rho=0.147442
2018-10-03 00:59:15,402 : INFO : PROGRESS: pass 0, at document #94000/100000
2018-10-03 00:59:16,441 : INFO : merging changes from 2000 documents into a model of 100000 documents
2018-10-03 00:59:16,480 : INFO : topic #0 (0.500): 0.049*"@realdonaldtrump" + 0.014*"#trump" + 0.010*"@hillaryclinton" + 0.008*"#imwithh" + 0.008*"@cnn" + 0.007*"#nevertrump" + 0.006*"tax" + 0.005*"@foxnew" + 0.005*"#hillaryclinton" + 0.004*""
2018-10-03 00:59:16,484 : INFO : topic #1 (0.500): 0.050*"@realdonaldtrump" + 0.027*"@hillaryclinton" + 0.014*"trump" + 0.007*"say" + 0.007*"get" + 0.006*"hillari" + 0.006*"go" + 0.006*"like" + 0.005*"vote" + 0.005*"u"
2018-10-03 00:59:16,486 : INFO : topic diff=0.125591, rho=0.145865
2018-

...

2018-10-03 01:02:30,768 : INFO : PROGRESS: pass 4, at document #6000/100000
2018-10-03 01:02:31,765 : INFO : merging changes from 2000 documents into a model of 100000 documents
2018-10-03 01:02:31,803 : INFO : topic #0 (0.500): 0.042*"@realdonaldtrump" + 0.012*"@hillaryclinton" + 0.011*"@cnn" + 0.010*"#trump" + 0.007*"tax" + 0.007*"#hillaryclinton" + 0.006*"#donaldtrump" + 0.006*"#nevertrump" + 0.006*"#imwithh" + 0.006*"@seanhann"
2018-10-03 01:02:31,806 : INFO : topic #1 (0.500): 0.053*"@realdonaldtrump" + 0.024*"@hillaryclinton" + 0.014*"trump" + 0.007*"say" + 0.007*"get" + 0.006*"#trump" + 0.006*"like" + 0.006*"hillari" + 0.006*"go" + 0.005*"u"
2018-10-03 01:02:31,807 : INFO : topic diff=0.118496, rho=0.134840
2018-10-03 01:02:31,812 : INFO : PROGRESS: pass 4, at document #8000/100000
2018-10-03 01:02:32,987 : INFO : merging changes from 2000 documents into a model of 100000 documents
2018-10-03 01:02:33,031 : INFO : topic #0 (0.500): 0.042*"@realdonaldtrump" + 0.012*"@hillaryclint

2018-10-03 01:02:45,060 : INFO : merging changes from 2000 documents into a model of 100000 documents
2018-10-03 01:02:45,097 : INFO : topic #0 (0.500): 0.042*"@realdonaldtrump" + 0.012*"@hillaryclinton" + 0.010*"@cnn" + 0.010*"#trump" + 0.007*"tax" + 0.006*"#donaldtrump" + 0.006*"#hillaryclinton" + 0.006*"#nevertrump" + 0.006*"#imwithh" + 0.005*"#neverhillari"
2018-10-03 01:02:45,100 : INFO : topic #1 (0.500): 0.052*"@realdonaldtrump" + 0.024*"@hillaryclinton" + 0.013*"trump" + 0.007*"get" + 0.007*"say" + 0.007*"#trump" + 0.006*"like" + 0.006*"hillari" + 0.006*"go" + 0.005*"u"
2018-10-03 01:02:45,103 : INFO : topic diff=0.126228, rho=0.134840
2018-10-03 01:02:45,106 : INFO : PROGRESS: pass 4, at document #30000/100000
2018-10-03 01:02:45,988 : INFO : merging changes from 2000 documents into a model of 100000 documents
2018-10-03 01:02:46,024 : INFO : topic #0 (0.500): 0.042*"@realdonaldtrump" + 0.012*"@hillaryclinton" + 0.011*"@cnn" + 0.010*"#trump" + 0.007*"tax" + 0.006*"#donaldtrump

2018-10-03 01:02:57,246 : INFO : topic #0 (0.500): 0.042*"@realdonaldtrump" + 0.013*"@hillaryclinton" + 0.011*"@cnn" + 0.010*"#trump" + 0.007*"tax" + 0.007*"#hillaryclinton" + 0.006*"#donaldtrump" + 0.006*"#imwithh" + 0.006*"#nevertrump" + 0.005*"@seanhann"
2018-10-03 01:02:57,249 : INFO : topic #1 (0.500): 0.051*"@realdonaldtrump" + 0.024*"@hillaryclinton" + 0.013*"trump" + 0.007*"say" + 0.007*"get" + 0.007*"#trump" + 0.006*"like" + 0.006*"hillari" + 0.006*"go" + 0.005*"know"
2018-10-03 01:02:57,251 : INFO : topic diff=0.128191, rho=0.134840
2018-10-03 01:02:57,253 : INFO : PROGRESS: pass 4, at document #52000/100000
2018-10-03 01:02:58,166 : INFO : merging changes from 2000 documents into a model of 100000 documents
2018-10-03 01:02:58,203 : INFO : topic #0 (0.500): 0.042*"@realdonaldtrump" + 0.012*"@hillaryclinton" + 0.011*"@cnn" + 0.010*"#trump" + 0.007*"tax" + 0.007*"#hillaryclinton" + 0.006*"#donaldtrump" + 0.006*"#imwithh" + 0.006*"#nevertrump" + 0.005*"@seanhann"
2018-10-03 01:

2018-10-03 01:03:11,220 : INFO : topic #1 (0.500): 0.052*"@realdonaldtrump" + 0.024*"@hillaryclinton" + 0.013*"trump" + 0.007*"get" + 0.007*"say" + 0.006*"#trump" + 0.006*"like" + 0.006*"go" + 0.006*"hillari" + 0.005*"vote"
2018-10-03 01:03:11,223 : INFO : topic diff=0.119302, rho=0.134840
2018-10-03 01:03:11,226 : INFO : PROGRESS: pass 4, at document #74000/100000
2018-10-03 01:03:12,327 : INFO : merging changes from 2000 documents into a model of 100000 documents
2018-10-03 01:03:12,372 : INFO : topic #0 (0.500): 0.042*"@realdonaldtrump" + 0.013*"@hillaryclinton" + 0.011*"@cnn" + 0.010*"#trump" + 0.007*"tax" + 0.007*"#hillaryclinton" + 0.006*"#donaldtrump" + 0.006*"#imwithh" + 0.005*"#maga" + 0.005*"#neverhillari"
2018-10-03 01:03:12,375 : INFO : topic #1 (0.500): 0.052*"@realdonaldtrump" + 0.024*"@hillaryclinton" + 0.013*"trump" + 0.007*"get" + 0.007*"say" + 0.006*"#trump" + 0.006*"like" + 0.006*"go" + 0.006*"hillari" + 0.005*"vote"
2018-10-03 01:03:12,379 : INFO : topic diff=0.1187

2018-10-03 01:03:24,749 : INFO : topic diff=0.119116, rho=0.134840
2018-10-03 01:03:24,754 : INFO : PROGRESS: pass 4, at document #96000/100000
2018-10-03 01:03:25,886 : INFO : merging changes from 2000 documents into a model of 100000 documents
2018-10-03 01:03:25,933 : INFO : topic #0 (0.500): 0.041*"@realdonaldtrump" + 0.013*"@hillaryclinton" + 0.010*"#trump" + 0.010*"@cnn" + 0.007*"tax" + 0.007*"#hillaryclinton" + 0.006*"#donaldtrump" + 0.006*"#nevertrump" + 0.006*"#neverhillari" + 0.006*"#maga"
2018-10-03 01:03:25,937 : INFO : topic #1 (0.500): 0.052*"@realdonaldtrump" + 0.025*"@hillaryclinton" + 0.013*"trump" + 0.007*"say" + 0.007*"get" + 0.007*"#trump" + 0.006*"like" + 0.006*"go" + 0.006*"hillari" + 0.005*"vote"
2018-10-03 01:03:25,939 : INFO : topic diff=0.127158, rho=0.134840
2018-10-03 01:03:25,945 : INFO : PROGRESS: pass 4, at document #98000/100000
2018-10-03 01:03:27,148 : INFO : merging changes from 2000 documents into a model of 100000 documents
2018-10-03 01:03:27,185 :

In [71]:
# Print 2 topics and describe then with 4 words.
topics = ldamodel.print_topics(num_topics=2, num_words=6)

i=0
for topic in topics:
    print ("Topic",i ,"->", topic)     
    i+=1


2018-10-03 01:04:00,657 : INFO : topic #0 (0.500): 0.041*"@realdonaldtrump" + 0.013*"@hillaryclinton" + 0.010*"@cnn" + 0.010*"#trump" + 0.007*"#hillaryclinton" + 0.007*"tax"
2018-10-03 01:04:00,661 : INFO : topic #1 (0.500): 0.052*"@realdonaldtrump" + 0.025*"@hillaryclinton" + 0.013*"trump" + 0.007*"say" + 0.007*"get" + 0.006*"#trump"


Topic 0 -> (0, '0.041*"@realdonaldtrump" + 0.013*"@hillaryclinton" + 0.010*"@cnn" + 0.010*"#trump" + 0.007*"#hillaryclinton" + 0.007*"tax"')
Topic 1 -> (1, '0.052*"@realdonaldtrump" + 0.025*"@hillaryclinton" + 0.013*"trump" + 0.007*"say" + 0.007*"get" + 0.006*"#trump"')
